# Building And Deploy Edge Optimized Object Detection Model (YoloV5s) To AWS Panorama : Avastus Pipelines

This notebook instance will act as the environment for setting up and triggering changes to our pipeline.

---

In [26]:
import boto3
import avastus_utils
from avastus_utils import communication
import pandas as pd
import json
import ast
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import os
import numpy
import re
import time
import sys
from yolov5 import utils
import glob
import joblib
import datetime
import warnings
import ipyplot
import shutil
import numpy as np

# from tqdm import tqdm
from time import strftime
from PIL import Image

# Sagemaker Imports
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import (Rule,
                                rule_configs,
                                ProfilerConfig, 
                                FrameworkProfile, 
                                DetailedProfilingConfig, 
                                DataloaderProfilingConfig, 
                                PythonProfilingConfig)

----
## Step 1:  Ground Truth Manifest to Yolo Format

  In the code cell below, we will take a completed Ground Truth Job, download its artifacts like the images and the manifest file, convert it to the Yolo Format. If you are interested in the YoloV5 format, please see this [Link](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data)

In [ ]:
### Provide Ground Truth Job Name
GT_Job_Name = 'wheel-job'

In [ ]:
labeling_job = avastus_utils.download_from_labeling_job(GT_Job_Name)
labeling_job.download_s3_data()
class_map = labeling_job.get_class_map()
labeling_job.split_images_test_train_valid()
labeling_job.create_train_test_valid()
lenclass = len(class_map)

----
## Step 2: Custom Training Container (ONE TIME RUN)

In [ ]:
iam_client = boto3.client('iam')
role=get_execution_role()
base_role_name=role.split('/')[-1]

In [ ]:
%%bash
cd ./ecr-repo
echo $(pwd)
container_name=yolov5-training-sagemaker
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

echo ${region}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${container_name}:1.10.0-gpu-py38"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${container_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${container_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin "763104351884.dkr.ecr.us-west-2.amazonaws.com")

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build -f Dockerfile -t ${fullname} .
# docker tag ${container_name} ${fullname}

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)
docker push ${fullname}

---
## Step 3:  Upload Data to S3 

We will utilize this notebook to perform some of the setup that will be required to trigger the first execution of our pipeline.   In this second step in our Machine Learning pipeline, we are going to simulate what would typically be the last step in an Analytics pipeline of creating datasets. 

To accomplish this, we will actually be uploading data from our local notebook instance (data can be found under /data/1-train/*) to S3.  In a typical scenario, this would be done through your analytics pipeline.  We will use the S3 bucket that was created through the CloudFormation template we launched at the beginning of the lab. You can validate the S3 bucket exists by:
  1. Going to the [S3 Service](https://s3.console.aws.amazon.com/s3/) inside the AWS Console
  2. Find the name of the S3 data bucket created by the CloudFormation template: mlops-data-*yourintials*-*randomid*
  
In the code cell below, we'll take a look at the training/test/validation datasets and then upload them to S3. 

   ### UPDATE THE BUCKET NAME BELOW BEFORE EXECUTING THE CELL

In [ ]:
# UPDATE THE NAME OF THE BUCKET TO MATCH THE ONE WE CREATED THROUGH THE CLOUDFORMATION TEMPLATE
# Example: mlops-data-jdd-df4d4850
#bucket = 'mlops-data-<yourinitials>-<generated id>'
bucket = 'mlops-data-avastus-8da27a80'

role = get_execution_role()
region = boto3.Session().region_name
sqs_client = boto3.client("sqs")
sagemaker_session = sagemaker.Session()

# Creating SQS Communication Channels

resp_1 = communication().create_queue('avastus-queue')
resp_2 = communication().create_queue('train-lambda-queue')
resp_3 = communication().create_queue('convert-onnx-queue')
resp_4 = communication().create_queue('optimize-model-queue')

### SEND MESSAGE TO THE SQS AVASTUS-QUEUE. THIS WILL BE BE READ BY THE TRAINING LAMBDA TO GET INFORMATION ABOUT THE S3 BUCKET AND FOLDER NAME 

In [ ]:
resp = communication().send_message(
            qurl = communication().get_queue_url('avastus-queue'),
            message = 'bucket:{}, data_folder:{}, epochs:{}'.format(bucket, GT_Job_Name, 20))

### SET CODE AND ARTIFACT PATHS. ALSO UPLOAD DATA TO THE S3 BUCKET

In [ ]:
code_location = f's3://{bucket}/avastus_yolov5/sm_codes'
output_path = f's3://{bucket}/avastus_yolov5/output' 
s3_log_path = f's3://{bucket}/avastus_yolov5/tf_logs'
source_code_path = f's3://{bucket}/avastus_yolov5/source_code'
s3_data_path = f's3://{bucket}/dataset/{GT_Job_Name}'

checkpoint_s3_uri = f's3://{bucket}/avastus_yolov5/checkpoints'
!aws s3 sync ./{GT_Job_Name} {s3_data_path} --quiet

----
## Step 3 : Customize Training

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate yolov5/data/data_sm.yaml
train: /opt/ml/input/data/yolov5_input/train/images
val: /opt/ml/input/data/yolov5_input/valid/images

nc: {lenclass}
names: {class_map}

In [ ]:
%%writetemplate yolov5/data_sm.yaml
train: /opt/ml/input/data/yolov5_input/train/images
val: /opt/ml/input/data/yolov5_input/valid/images

nc: {lenclass}
names: {class_map}

In [ ]:
%%writetemplate data_sm.yaml
train: /opt/ml/input/data/yolov5_input/train/images
val: /opt/ml/input/data/yolov5_input/valid/images

nc: {lenclass}
names: {class_map}

In [ ]:
os.system("tar -zcvf yolov5.tar.gz ./yolov5")
os.system('aws s3 cp yolov5.tar.gz s3://{}/avastus_yolov5/source_code/yolov5.tar.gz --quiet'.format(bucket))
os.system('rm -r yolov5.tar.gz')

---
## Step 4:  Commit Training Code To Trigger Pipeline Build

In this step, we are going to trigger an execution of the pipeline by committing our training code to the CodeCommit repository that was setup as part of the CloudFormation stack.  The CodeCommit repository created was associated with this SageMaker Notebook Instance via a setting in the CloudFormation Stack using the [SageMaker Notebook Instance Git Association](https://docs.aws.amazon.com/sagemaker/latest/dg/nbi-git-repo.html) feature.

The pipeline is currently setup to trigger on a commit to the master branch; however, this should be adjusted in a real-world scenario based on your branching strategy. 

The CodeCommit repository created can be viewed by:
  1. Going to the [CodeCommit Service](https://console.aws.amazon.com/codesuite/codecommit/repositories) inside the AWS Console
  2. Find the name of the repository created by the CloudFormation template: mlops-codecommit-byo-*yourinitials*
  
**UPDATE** Ensure you update the cell below where noted prior to executing 

In [ ]:
# View the CodeCommit repository -
# This Git integration was configured as part of the creation of the notebook instance in the CloudFormation stack.

# The following will return the CodeCommit repository that has been configured with this notebook and will be used 
# for the source control repository during this workshop. 
!rm -r {GT_Job_Name}

# Ensure remote repo is setup
!git remote -v

!git config --global user.name "Surya Kari"
!git config --global user.email karisury@amazon.com

### Commit training code to the CodeCommit repository to trigger the execution of the CodePipeline

In [ ]:
!git pull
!git add ./model-code/*
!git add ./lambda-code/*
!git add ./ecr-repo/*
!git add ./yolov5
!git add *
!git commit -m "Avastus Pipelines : Wheel Labeling Job Test 2 with ONNX: 20 Epochs"
!git push origin master

----

## Step 4:  Monitor CodePipeline Execution

The code above will trigger the execution of your CodePipeline. You can monitor progress of the pipeline execution in the [CodePipeline dashboard](https://console.aws.amazon.com/codesuite/codepipeline/pipelines).

You can also validate that your code is now committed to the CodeCommit repository in the [CodeCommit dashboard](https://console.aws.amazon.com/codesuite/codecommit/repositories)

As the pipeline is executing information is being logged to [Cloudwatch logs](https://console.aws.amazon.com/cloudwatch/logs).  Explore the logs for your Lambda functions (/aws/lambda/MLOps-BYO*) as well as output logs from SageMaker (/aws/sagemaker/*)


Note: It will take awhile to execute all the way through the pipeline.  Please don't proceed to the next step until the last stage is shows **'succeeded'**

Because we may not want rebuild the training/inference container image in the case where we only want to retrain the model, you could optionally create a separate retraining pipeline that excludes the rebuild of the image.  Depending on what you are using for orchestration across your pipeline, you can accomplish this through a single or multiple pipelines. 


After executing the cell above, you will see a new trigger for pipeline execution when you click on the link to your [pipeline](https://console.aws.amazon.com/codesuite/codepipeline/pipelines)


----

## Step 5:  Output Of Trained Model

In [ ]:
messages = communication().receive_message(qurl = communication().get_queue_url('train-lambda-queue'))

In [ ]:
estimator = sagemaker.estimator.Estimator.attach(messages[0][0].split(':')[-1].replace('"',''))

In [ ]:
artifacts_dir = estimator.model_data.replace('model.tar.gz', '')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

In [ ]:
model_dir = './model'
!rm -rf $model_dir

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

!aws s3 cp {artifacts_dir}model.tar.gz {model_dir}/model.tar.gz
!tar -xvzf {model_dir}/model.tar.gz -C {model_dir}

In [ ]:
sys.path.append(f'{os.getcwd()}/yolov5')
#display = utils.notebook_init()  # checks

In [ ]:
images = []
for img_path in glob.glob('model/*/*'):
    if img_path.split('.')[-1] in ['jpg','png']:
        images.append(mpimg.imread(img_path))

In [ ]:
ipyplot.plot_images(images, max_images=5, img_width=450)